In [7]:
import gradio as gr
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer

df = pd.read_csv("../data/processed/original_merged_data.csv")

sentence_transformer = SentenceTransformer("../models/sales_sentence_transformer")

# reconnect to persistent chromadb client
client = chromadb.PersistentClient(path="../chroma_store")

# get account collection
account_collection = client.get_collection("sales_accounts")

def get_result_from_query(query, collection, model, n_results=10):
    # Encode the query using the sentence transformer
    query_emb = model.encode(query, convert_to_numpy=True)

    # Query Chroma using the embedding
    res = collection.query(
        query_embeddings=[query_emb],
        n_results=n_results,
        include=["documents", "metadatas", "distances"],
    )
    return res

def chat_fn(message, history):
    res = get_result_from_query(
        message,
        account_collection,
        sentence_transformer,
        n_results=10
    )

    ids = res["ids"][0]
    matches = df[df["opportunity_id"].isin(ids)].copy()
    matches["__rank"] = matches["opportunity_id"].apply(lambda x: ids.index(x))
    matches = matches.sort_values("__rank").drop(columns="__rank")

    # keep only the most useful columns for display
    cols = [
        "opportunity_id", "sales_agent", "product", "account",
        "deal_stage", "engage_date", "close_date",
        "revenue", "employees", "office_location"
    ]
    matches = matches[cols]

    table_md = matches.to_markdown(index=False)

    response = (
        f"Here are the opportunities I found that match your query:\n\n"
        f"{table_md}\n\n"
        f"Ask another question or refine your query!"
    )

    return response

demo = gr.ChatInterface(
    chat_fn,
    title="Salesforce 2A",
    description="Hi! I can help you explore Salesforce CRM opportunities and accounts. What would you like to know?"
)

demo.launch(share=True)


/opt/anaconda3/lib/python3.12/site-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7865


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on public URL: https://d050a6428b9dc86d10.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
